Setup Environment

In [12]:
# Clear the R environment and set working directory
rm(list = ls())
setwd("/Users/jan/Dropbox/UP_EPQM/2222/MA/powerlinemonsters")
path <- getwd()
path 
options(warn=-1)

[1] "/Users/jan/Dropbox/UP_EPQM/2222/MA/powerlinemonsters"

Load packages

In [13]:
# Install and load additional packages needed
#install.packages("did")
library(did)
library(rio)
library(ggplot2)
library(ggpubr)
library(hash)
.libPaths()

[1] "/Library/Frameworks/R.framework/Versions/4.2-arm64/Resources/library"

DiD for BTW no controls

In [14]:
# read in the data set
btw <- import(sprintf("%s/data/btw_treat.csv", path))

In [15]:
parties <- c('Union', 'SPD', 'FDP', 'Linke', 'Grüne', 'Andere')
treatments_dict <- hash('treatment_0'='Direct Line', 'treatment_15'='Within 0-15 km', 'treatment_30'='Within 15-30 km', 'treatment_50'='Within 30-50 km')
treatments <- keys(treatments_dict)
results  <- list()
# set up Fig Number
n <- 1
for(party in parties){
  figures  <- list()
  for(treatment in treatments){
    result <- att_gt(yname = party,
                  gname = treatment,
                  idname = 'AGS',
                  panel = TRUE,
                  tname = 'year',
                  xformla = ~ 1,
                  data = btw,
                  est_method = 'reg',
                  anticipation = 0,
                  control_group = 'nevertreated',
                  clustervars = c('AGS'),
                  bstrap = TRUE,
                  cband = TRUE,
                  allow_unbalanced_panel = TRUE,
                  
    )
    results[[length(results)+1]] <- result
    # calculate ATT
    att <- aggte(result, type = "group", bstrap = TRUE, clustervars = c('AGS'))
    # plot results
    result_fig <- ggdid(result, 
              ylim = c(floor(min(result$att - result$se * 2.345 - 1)), ceiling(max(result$att + result$se * 2.345 + 1))),
              ncol = 3, ax_text_size = 8, grtitle='', title_size = 10, legend=F) +
              labs(caption = sprintf('Overall ATT (SE): %.3f (%.3f) \n Wald-p: %.3f; Municipalities: %s \n' ,
              att$overall.att, att$overall.se, result$Wpval, format(format(result$n, big.mark=","), big.mark=","))) +
              theme(plot.caption = element_text(hjust=0.5, size=8),
              panel.background = element_rect(fill='transparent'),
              plot.background = element_rect(fill='transparent', color=NA),
              legend.background = element_rect(fill='transparent'),
              legend.box.background = element_rect(fill='transparent'),
              plot.margin=unit(c(0,0,0,0),"cm"))
    # Get caption for ATT plot
    cap = ''
      for(i in 1:length(att$DIDparams$glist)) {
        cap <- sprintf('%s ATT (SE): %.3f (%.3f)\n%s', att$DIDparams$glist[[i]], att$att.egt[[i]], att$se.egt[[i]], cap)
      }
    # Plot ATT
    att_fig <- ggdid(att, legend=F, x_lab='', ax_text_size = 8, ylab='', title='ATT by group', title_size = 10) + 
    labs(caption = cap) + 
              theme(plot.caption = element_text(hjust=0.5, size=8), 
              axis.title.x = element_blank(),
              axis.text.y = element_text(angle=90, size=6, hjust=0.5),
              panel.background = element_rect(fill='transparent'),
              plot.background = element_rect(fill='transparent', color=NA),
              legend.background = element_rect(fill='transparent'),
              legend.box.background = element_rect(fill='transparent'),
              plot.margin=unit(c(0,0,0,0),"cm"))
    # Combine and annotate Group and ATT plot
    # Get number of results plots
    nr <- length(unique(result_fig$data$grtitle))
    combined_fig <- ggarrange(result_fig, att_fig, widths = c(nr, 1)) + 
                    theme(plot.margin=unit(c(0,0,0,0),"cm"))
    combined_fig_anno <- annotate_figure(combined_fig,
                  top = text_grob(sprintf('Treatment: %s', treatments_dict[[treatment]])))
    figures[[length(figures)+1]] <- combined_fig_anno
  }
  # Combine Figures for all treatments
  arranged_fig <- ggarrange(plotlist=figures, nrow = 4, ncol = 1, common.legend = TRUE)
  # get legend
  result_fig <- ggdid(result) +
              theme(
              legend.background = element_rect(fill='transparent', color='transparent'),
              legend.box.background = element_rect(fill='transparent', color='transparent'),
              plot.margin=unit(c(0,0,0,0),"cm"))
  leg <- get_legend(result_fig)
  # add legend
  arranged_fig_leg <- ggarrange(arranged_fig, leg, nrow = 2, ncol = 1, heights = c(10, 1))
  # Annotate final figure
  final_fig <- annotate_figure(arranged_fig_leg,
                  top = text_grob(sprintf("BTW - Effect on %s's vote share", party), face = "bold", size = 14),
                  bottom = text_grob('Control Group: Never Treated. Estimation Method: Regression \n Standard Errors clustered on the municipality level.', size = 10))
  # Add Figure number
  final_fig <- annotate_figure(final_fig, top = text_grob(sprintf("Fig.: R%s", n), face = "bold", size = 10))
  # Save
  ggsave(sprintf('%s_%s.png', n, party), plot = final_fig, path = sprintf('%s/figures/R/BTW/pooled', path), units = 'cm', width = 21, height = 21, dpi="print")
  print(party)
  n <- n + 1
}

[1] "Union"
[1] "SPD"
[1] "FDP"
[1] "Linke"
[1] "Grüne"
[1] "Andere"


DiD for BTW with controls

In [16]:
btw_c <- import(sprintf("%s/data/btw_control.csv", path))

In [17]:
parties <- c('Union', 'SPD', 'FDP', 'Linke', 'Grüne', 'Andere')
treatments_dict <- hash('treatment_0'='Direct Line', 'treatment_15'='Within 0-15 km', 'treatment_30'='Within 15-30 km', 'treatment_50'='Within 30-50 km')
treatments <- keys(treatments_dict)
results  <- list()
n <- 7
for(party in parties){
  figures  <- list()
  for(treatment in treatments){
    result <- att_gt(yname = party,
                  gname = treatment,
                  idname = 'AGS',
                  panel = TRUE,
                  tname = 'year',
                  xformla = ~ east + south + pop_density + unemployed + female + avg_age,
                  data = btw_c,
                  est_method = 'reg',
                  anticipation = 0,
                  control_group = 'nevertreated',
                  clustervars = c('AGS'),
                  bstrap = TRUE,
                  cband = TRUE,
                  allow_unbalanced_panel = TRUE,
                  
    )
    results[[length(results)+1]] <- result
    # calculate ATT
    att <- aggte(result, type = "group", bstrap = TRUE, clustervars = c('AGS'))
    # plot results
    result_fig <- ggdid(result, 
              ylim = c(floor(min(result$att - result$se * 2.345 - 1)), ceiling(max(result$att + result$se * 2.345 + 1))),
              ncol = 3, ax_text_size = 8, grtitle='', title_size = 10, legend=F) +
              labs(caption = sprintf('Overall ATT (SE): %.3f (%.3f) \n Wald-p: %.3f; Municipalities: %s \n' ,
              att$overall.att, att$overall.se, result$Wpval, format(result$n, big.mark=","))) +
              theme(plot.caption = element_text(hjust=0.5, size=8),
              panel.background = element_rect(fill='transparent'),
              plot.background = element_rect(fill='transparent', color=NA),
              legend.background = element_rect(fill='transparent'),
              legend.box.background = element_rect(fill='transparent'),
              plot.margin=unit(c(0,0,0,0),"cm"))
    # Get caption for ATT plot
    cap = ''
      for(i in 1:length(att$DIDparams$glist)) {
        cap <- sprintf('%s ATT (SE): %.3f (%.3f)\n%s', att$DIDparams$glist[[i]], att$att.egt[[i]], att$se.egt[[i]], cap)
      }
    # Plot ATT
    att_fig <- ggdid(att, legend=F, x_lab='', ax_text_size = 8, ylab='', title='ATT by group', title_size = 10) + 
    labs(caption = cap) + 
              theme(plot.caption = element_text(hjust=0.5, size=8), 
              axis.title.x = element_blank(),
              axis.text.y = element_text(angle=90, size=6, hjust=0.5),
              panel.background = element_rect(fill='transparent'),
              plot.background = element_rect(fill='transparent', color=NA),
              legend.background = element_rect(fill='transparent'),
              legend.box.background = element_rect(fill='transparent'),
              plot.margin=unit(c(0,0,0,0),"cm"))
    # Combine and annotate Group and ATT plot
    # Get number of results plots
    nr <- length(unique(result_fig$data$grtitle))
    combined_fig <- ggarrange(result_fig, att_fig, widths = c(nr, 1)) + 
                    theme(plot.margin=unit(c(0,0,0,0),"cm"))
    combined_fig_anno <- annotate_figure(combined_fig,
                  top = text_grob(sprintf('Treatment: %s', treatments_dict[[treatment]])))
    figures[[length(figures)+1]] <- combined_fig_anno
  }
  # Combine Figures for all treatments
  arranged_fig <- ggarrange(plotlist=figures, nrow = 4, ncol = 1, common.legend = TRUE)
  # get legend
  result_fig <- ggdid(result) +
              theme(
              legend.background = element_rect(fill='transparent', color='transparent'),
              legend.box.background = element_rect(fill='transparent', color='transparent'),
              plot.margin=unit(c(0,0,0,0),"cm"))
  leg <- get_legend(result_fig)
  # add legend
  arranged_fig_leg <- ggarrange(arranged_fig, leg, nrow = 2, ncol = 1, heights = c(10, 1))
  # Annotate final figure
  final_fig <- annotate_figure(arranged_fig_leg,
                  top = text_grob(sprintf("BTW - Effect on %s's vote share", party), face = "bold", size = 14),
                  bottom = text_grob('Control Group: Never Treated. Estimation Method: Regression. \n Standard Errors clustered on the municipality level. \n Included Controls: Population Density, Share of Unemployed, Share of Females, Average Age, \n Dummies for East and South Germany.', size = 10))
  # Add Figure number
  final_fig <- annotate_figure(final_fig, top = text_grob(sprintf("Fig.: R%s", n), face = "bold", size = 10))
  # Save
  ggsave(sprintf('%s_%s.png', n, party), plot = final_fig, path = sprintf('%s/figures/R/BTW/control', path), units = 'cm', width = 21, height = 21, dpi="print")
  print(party)
  n <- n + 1
}

[1] "Union"
[1] "SPD"
[1] "FDP"
[1] "Linke"
[1] "Grüne"
[1] "Andere"


DiD for LTW no controls by state

In [18]:
# read in the data set
ltw <- import(sprintf("%s/data/ltw_treat.csv", path))

In [19]:
parties <- c('Union', 'SPD', 'FDP', 'Linke', 'Grüne', 'Andere')
treatments_dict <- hash('treatment_0'='Direct Line', 'treatment_15'='Within 0-15 km', 'treatment_30'='Within 15-30 km', 'treatment_50'='Within 30-50 km')
states_dict <- hash('NI'='Lower Saxony', 'NW'='North-Rhine Westphalia', 'HE'='Hesse', 'BY'='Bavaria')
treatments <- keys(treatments_dict)
states <- keys(states_dict)
results  <- list()
n <- 14
for (state in states){
  for(party in parties){
    figures  <- list()
    reg_data <- subset(ltw, Land == state)
    for(treatment in treatments){
      result <- att_gt(yname = party,
                    gname = treatment,
                    idname = 'AGS',
                    panel = TRUE,
                    tname = 'year',
                    xformla = ~ 1,
                    data = reg_data,
                    est_method = 'reg',
                    anticipation = 0,
                    control_group = 'notyettreated',
                    clustervars = c('AGS'),
                    bstrap = TRUE,
                    cband = TRUE,
                    allow_unbalanced_panel = TRUE,
                    
      )
    results[[length(results)+1]] <- result
    # calculate ATT
    att <- aggte(result, type = "group", bstrap = TRUE, clustervars = c('AGS'))
    # plot results
    result_fig <- ggdid(result, 
              ylim = c(floor(min(result$att - result$se * 2.345 - 1)), ceiling(max(result$att + result$se * 2.345 + 1))),
              ncol = 3, ax_text_size = 8, grtitle='', title_size = 10, legend=F) +
              labs(caption = sprintf('Overall ATT (SE): %.3f (%.3f) \n Wald-p: %.3f; Municipalities: %s \n' ,
              att$overall.att, att$overall.se, result$Wpval, format(result$n, big.mark=","))) +
              theme(plot.caption = element_text(hjust=0.5, size=8),
              panel.background = element_rect(fill='transparent'),
              plot.background = element_rect(fill='transparent', color=NA),
              legend.background = element_rect(fill='transparent'),
              legend.box.background = element_rect(fill='transparent'),
              plot.margin=unit(c(0,0,0,0),"cm"))
    # Get caption for ATT plot
    cap = ''
      for(i in 1:length(att$DIDparams$glist)) {
        cap <- sprintf('%s ATT (SE): %.3f (%.3f)\n%s', att$DIDparams$glist[[i]], att$att.egt[[i]], att$se.egt[[i]], cap)
    # Plot ATT
    att_fig <- ggdid(att, legend=F, x_lab='', ax_text_size = 8, ylab='', title='ATT by group', title_size = 10) + 
    labs(caption = cap) + 
              theme(plot.caption = element_text(hjust=0.5, size=8), 
              axis.title.x = element_blank(),
              axis.text.y = element_text(angle=90, size=6, hjust=0.5),
              panel.background = element_rect(fill='transparent'),
              plot.background = element_rect(fill='transparent', color=NA),
              legend.background = element_rect(fill='transparent'),
              legend.box.background = element_rect(fill='transparent'),
              plot.margin=unit(c(0,0,0,0),"cm"))
    # Combine and annotate Group and ATT plot
    # Get number of results plots
    nr <- length(unique(result_fig$data$grtitle))
    combined_fig <- ggarrange(result_fig, att_fig, widths = c(nr, 1)) + 
                    theme(plot.margin=unit(c(0,0,0,0),"cm"))
    combined_fig_anno <- annotate_figure(combined_fig,
                  top = text_grob(sprintf('Treatment: %s', treatments_dict[[treatment]])))
    figures[[length(figures)+1]] <- combined_fig_anno
    }
    # Combine Figures for all treatments
    arranged_fig <- ggarrange(plotlist=figures, nrow = 4, ncol = 1, common.legend = TRUE)
    # get legend
    result_fig <- ggdid(result) +
                theme(
                legend.background = element_rect(fill='transparent', color='transparent'),
                legend.box.background = element_rect(fill='transparent', color='transparent'),
                plot.margin=unit(c(0,0,0,0),"cm"))
    leg <- get_legend(result_fig)
    # add legend
    arranged_fig_leg <- ggarrange(arranged_fig, leg, nrow = 2, ncol = 1, heights = c(10, 1))
    final_fig <- annotate_figure(arranged_fig_leg,
                    top = text_grob(sprintf("LTW %s - Effect on %s's vote share", states_dict[[state]], party), face = "bold", size = 14),
                    bottom = text_grob('Control Group: Not-yet Treated. Estimation Method: Regression \n Standard Errors clustered on the municipality level.', size = 10),)
    # Add Figure number
    final_fig <- annotate_figure(final_fig, top = text_grob(sprintf("Fig.: R%s", n), face = "bold", size = 10))
    # Save
    ggsave(sprintf('%s_%s_%s.png', n, state, party), plot = final_fig, path = sprintf('%s/figures/R/LTW/by_state', path), units = 'cm', width = 21, height = 29.7, dpi="print")
    print(sprintf('%s: %s', state, party))
    n <- n + 2
  }
}

ERROR: Error in parse(text = x, srcfile = src): <text>:90:0: unexpected end of input
88:   }
89: }
   ^


DiD for LTW controls by state

In [ ]:
ltw_c <- import(sprintf("%s/data/ltw_control.csv", path))

In [ ]:
parties <- c('Union', 'SPD', 'FDP', 'Linke', 'Grüne', 'Andere')
treatments_dict <- hash('treatment_0'='Direct Line', 'treatment_15'='Within 0-15 km', 'treatment_30'='Within 15-30 km', 'treatment_50'='Within 30-50 km')
states_dict <- hash('NI'='Lower Saxony', 'NW'='North-Rhine Westphalia', 'HE'='Hesse', 'BY'='Bavaria')
treatments <- keys(treatments_dict)
states <- keys(states_dict)
results  <- list()
n <- 15
for (state in states){
  for(party in parties){
    figures  <- list()
    reg_data <- subset(ltw_c, Land == state)
    for(treatment in treatments){
      result <- att_gt(yname = party,
                    gname = treatment,
                    idname = 'AGS',
                    panel = TRUE,
                    tname = 'year',
                    xformla = ~ pop_density + unemployed + avg_age + female,
                    data = reg_data,
                    est_method = 'reg',
                    anticipation = 0,
                    control_group = 'notyettreated',
                    clustervars = c('AGS'),
                    bstrap = TRUE,
                    cband = TRUE,
                    allow_unbalanced_panel = TRUE,
                    
      )
    results[[length(results)+1]] <- result
    # calculate ATT
    att <- aggte(result, type = "group", bstrap = TRUE, clustervars = c('AGS'))
    # plot results
    result_fig <- ggdid(result, 
              ylim = c(floor(min(result$att - result$se * 2.345 - 1)), ceiling(max(result$att + result$se * 2.345 + 1))),
              ncol = 3, ax_text_size = 8, grtitle='', title_size = 10, legend=F) +
              labs(caption = sprintf('Overall ATT (SE): %.3f (%.3f) \n Wald-p: %.3f; Municipalities: %s \n' ,
              att$overall.att, att$overall.se, result$Wpval, format(result$n, big.mark=","))) +
              theme(plot.caption = element_text(hjust=0.5, size=8),
              panel.background = element_rect(fill='transparent'),
              plot.background = element_rect(fill='transparent', color=NA),
              legend.background = element_rect(fill='transparent'),
              legend.box.background = element_rect(fill='transparent'),
              plot.margin=unit(c(0,0,0,0),"cm"))
    # Get caption for ATT plot
    cap = ''
      for(i in 1:length(att$DIDparams$glist)) {
        cap <- sprintf('%s ATT (SE): %.3f (%.3f)\n%s', att$DIDparams$glist[[i]], att$att.egt[[i]], att$se.egt[[i]], cap)
      }
    # Plot ATT
    att_fig <- ggdid(att, legend=F, x_lab='', ax_text_size = 8, ylab='', title='ATT by group', title_size = 10) + 
    labs(caption = cap) + 
              theme(plot.caption = element_text(hjust=0.5, size=8), 
              axis.title.x = element_blank(),
              axis.text.y = element_text(angle=90, size=6, hjust=0.5),
              panel.background = element_rect(fill='transparent'),
              plot.background = element_rect(fill='transparent', color=NA),
              legend.background = element_rect(fill='transparent'),
              legend.box.background = element_rect(fill='transparent'),
              plot.margin=unit(c(0,0,0,0),"cm"))
    # Combine and annotate Group and ATT plot
    # Get number of results plots
    nr <- length(unique(result_fig$data$grtitle))
    combined_fig <- ggarrange(result_fig, att_fig, widths = c(nr, 1)) + 
                    theme(plot.margin=unit(c(0,0,0,0),"cm"))
    combined_fig_anno <- annotate_figure(combined_fig,
                  top = text_grob(sprintf('Treatment: %s', treatments_dict[[treatment]])))
    figures[[length(figures)+1]] <- combined_fig_anno
    }
    # Combine Figures for all treatments
    arranged_fig <- ggarrange(plotlist=figures, nrow = 4, ncol = 1, common.legend = TRUE)
    # get legend
    result_fig <- ggdid(result) +
                theme(
                legend.background = element_rect(fill='transparent', color='transparent'),
                legend.box.background = element_rect(fill='transparent', color='transparent'),
                plot.margin=unit(c(0,0,0,0),"cm"))
    leg <- get_legend(result_fig)
    # add legend
    arranged_fig_leg <- ggarrange(arranged_fig, leg, nrow = 2, ncol = 1, heights = c(10, 1))
    final_fig <- annotate_figure(arranged_fig_leg,
                    top = text_grob(sprintf("LTW %s - Effect on %s's vote share", states_dict[[state]], party), face = "bold", size = 14),
                    bottom = text_grob('Control Group: Not-yet Treated. Estimation Method: Regression. \n Standard Errors clustered on the municipality level. \n Included Controls: Population Density, Share of Unemployed, Share of Females, Average Age.', size = 10))
    # Add Figure number
    final_fig <- annotate_figure(final_fig, top = text_grob(sprintf("Fig.: R%s", n), face = "bold", size = 10))
    # Save
    ggsave(sprintf('%s_%s_%s.png', n, state, party), plot = final_fig, path = sprintf('%s/figures/R/LTW/by_state_controls', path), units = 'cm', width = 21, height = 29.7, dpi="print")
    print(sprintf('%s: %s', state, party))
    n <- n + 2
  }
}

DiD for BTW with controls by state

In [ ]:
btw_c <- import(sprintf("%s/data/btw_control.csv", path))

In [ ]:
parties <- c('Union', 'SPD', 'FDP', 'Linke', 'Grüne', 'Andere')
treatments_dict <- hash('treatment_0'='Direct Line', 'treatment_15'='Within 0-15 km', 'treatment_30'='Within 15-30 km', 'treatment_50'='Within 30-50 km')
states_dict <- hash('SH'='Schleswig-Holstein', 'NI'='Lower Saxony', 'NW'='North-Rhine Westphalia', 'BW'='Baden-Württemberg', 
'BY'='Bavaria', 'BB'='Brandenburg', 'MV'='Mecklenburg-Vorpommern', 'ST'='Saxony-Anhalt') # all states - those never treated
treatments <- keys(treatments_dict)
states <- keys(states_dict)
results  <- list()
n <- 13
for (state in states){
  for(party in parties){
    reg_data <- subset(btw_c, Land == state)
    figures  <- list()
    for(treatment in treatments){
      result <- att_gt(yname = party,
                    gname = treatment,
                    idname = 'AGS',
                    panel = TRUE,
                    tname = 'year',
                    xformla = ~ pop_density + unemployed + female + avg_age,
                    data = reg_data,
                    est_method = 'reg',
                    anticipation = 0,
                    control_group = 'notyettreated',
                    clustervars = c('AGS'),
                    bstrap = TRUE,
                    cband = TRUE,
                    allow_unbalanced_panel = TRUE,
                    
      )
    results[[length(results)+1]] <- result
    # calculate ATT
    att <- aggte(result, type = "group", bstrap = TRUE, clustervars = c('AGS'))
    # plot results
    result_fig <- ggdid(result, 
              ylim = c(floor(min(result$att - result$se * 2.345 - 1)), ceiling(max(result$att + result$se * 2.345 + 1))),
              ncol = 3, ax_text_size = 8, grtitle='', title_size = 10, legend=F) +
              labs(caption = sprintf('Overall ATT (SE): %.3f (%.3f) \n Wald-p: %.3f; Municipalities: %s \n' ,
              att$overall.att, att$overall.se, result$Wpval, format(result$n, big.mark=","))) +
              theme(plot.caption = element_text(hjust=0.5, size=8),
              panel.background = element_rect(fill='transparent'),
              plot.background = element_rect(fill='transparent', color=NA),
              legend.background = element_rect(fill='transparent'),
              legend.box.background = element_rect(fill='transparent'),
              plot.margin=unit(c(0,0,0,0),"cm"))
    # Get caption for ATT plot
    cap = ''
      for(i in 1:length(att$DIDparams$glist)) {
        cap <- sprintf('%s ATT (SE): %.3f (%.3f)\n%s', att$DIDparams$glist[[i]], att$att.egt[[i]], att$se.egt[[i]], cap)
      }
    # Plot ATT
    att_fig <- ggdid(att, legend=F, x_lab='', ax_text_size = 8, ylab='', title='ATT by group', title_size = 10) + 
              labs(caption = cap) +
              theme(plot.caption = element_text(hjust=0.5, size=8), 
              axis.title.x = element_blank(),
              axis.text.y = element_text(angle=90, size=6, hjust=0.5),
              panel.background = element_rect(fill='transparent'),
              plot.background = element_rect(fill='transparent', color=NA),
              legend.background = element_rect(fill='transparent'),
              legend.box.background = element_rect(fill='transparent'),
              plot.margin=unit(c(0,0,0,0),"cm"))
    # Combine and annotate Group and ATT plot
    # Get number of results plots
    nr <- length(unique(result_fig$data$grtitle))
    combined_fig <- ggarrange(result_fig, att_fig, widths = c(nr, 1)) + 
                    theme(plot.margin=unit(c(0,0,0,0),"cm"))
    combined_fig_anno <- annotate_figure(combined_fig,
                  top = text_grob(sprintf('Treatment: %s', treatments_dict[[treatment]])))
    figures[[length(figures)+1]] <- combined_fig_anno
    }
    # Combine Figures for all treatments
    arranged_fig <- ggarrange(plotlist=figures, nrow = 4, ncol = 1, common.legend = TRUE)
    # get legend
    result_fig <- ggdid(result) +
                theme(
                legend.background = element_rect(fill='transparent', color='transparent'),
                legend.box.background = element_rect(fill='transparent', color='transparent'),
                plot.margin=unit(c(0,0,0,0),"cm"))
    leg <- get_legend(result_fig)
    # add legend
    arranged_fig_leg <- ggarrange(arranged_fig, leg, nrow = 2, ncol = 1, heights = c(10, 1))
    # Annotate final figure
    final_fig <- annotate_figure(arranged_fig_leg,
                    top = text_grob(sprintf("BTW in %s - Effect on %s's vote share", states_dict[[state]], party), face = "bold", size = 14),
                    bottom = text_grob('Control Group: Not-yet Treated. Estimation Method: Regression. \n Standard Errors clustered on the municipality level. \n Included Controls: Population Density, Share of Unemployed, Share of Females, Average Age.', size = 10))
    # Add Figure number
    final_fig <- annotate_figure(final_fig, top = text_grob(sprintf("Fig.: R%s", n), face = "bold", size = 10))
    # Save
    ggsave(sprintf('%s_%s_%s.png', n, state, party), plot = final_fig, path = sprintf('%s/figures/R/BTW/by_state', path), units = 'cm', width = 21, height = 29.7, dpi="print")
    print(sprintf('%s: %s', state, party))
    n <- n + 1
  }
}